# 利用py注解实现创建sql

本文只提供了最基本的定制sql的注解，但是实际中，你可以通过这些基本注解，再定义自己的注解，从而实现更丰富的功能。

In [63]:
from imp import reload
from sql_builder import *

reload(sql_builder)

<module 'sql_builder' from '/Users/liuning11/project/py-sql-builder/sql_builder.py'>

# 表

- Persons 表

Id_P	| LastName	| FirstName	| Address	| City
---|---|---|---|---
1	| Adams	| John	| Oxford | Street London
- | - | - | - | - 

- Orders 表

Id_O	| OrderNo	| Id_P
---|---|---
1	| 77895	| 3
- | - | -


# 测试Select和SubQuery

In [64]:
def test_Select_and_SubQuery():

    @SubQuery(field=['Id_P', 'concat(FirstName,LastName) as name'], alias='p')
    @Select(field=['Id_P', 'LastName', 'FirstName'], f=["1=1"])
    def table():
        return 'Persons'

    s = table()
    print('s:   {}'.format(s))

In [65]:
test_Select_and_SubQuery()

s:   select Id_P,concat(FirstName,LastName) as name from (select Id_P,LastName,FirstName from Persons WHERE 1=1) p


# 测试AggregateSel

In [66]:
def test_AggregateSel():

    @AggregateSel(field=['Id_P','count(*)'], f=['1=1'], group=['Id_P'])
    def Persons():
        return 'Orders'

    s = Persons()
    print('join:  {}'.format(s))

In [67]:
test_AggregateSel()

join:  select Id_P,count(*) from Orders WHERE 1=1 GROUP BY Id_P


# 测试Join

In [39]:
def test_Join():

    @Select(field=['*'], f=[])
    def Persons():
        return 'Persons'

    @Select(field=['*'], f=[])
    def Orders():
        return 'Orders'

    sql_p = Persons()
    sql_o = Orders()
    print('filter person:{}'.format(sql_p))
    print('filter order: {}'.format(sql_o))

    @Join(t="eq", before=sql_o, after=sql_p, 
          field=['t1.OrderNo','concat(t2.FirstName, t2.LastName) as name'], 
          on=['t1.Id_P=t2.Id_P'],before_alias="t1", after_alias="t2")
    def join(before_field=[], after_field=[]):
        return {
            "before_field": before_field,
            "after_field": after_field
        }

    s3 = join()
    print('join:  {}'.format(s3))

In [40]:
test_Join()

filter person:select * from Persons 
filter order: select * from Orders 
join:  
    select t1.OrderNo,concat(t2.FirstName, t2.LastName) as name from (select * from Orders ) t1
    join (select * from Persons ) t2 on(t1.Id_P=t2.Id_P)
    


# 测试SelectMap和Map

In [41]:
def test_SelectMap_and_Map():

    @SelectMap(
        field=['id', 'start_date', 'len', "regexp_replace(regexp_replace(y, '\\\\]', ''), '\\\\[', '') as y"],
        field_map=['id', 'start_date', 'len', 'split(y) as y'], alias="t1",
        f=["type='ttt'", "and", "key='sales'"])
    def select_map(table):
        return {"table": table}

    s1 = select_map(table="mytable")
    print('s1:  {}'.format(s1))

    @Map(mapper=['id', 'start_date', 'len', "concat(',', d1, d2)"], alias="t2")
    @Map(mapper=['id', 'start_date', 'len', 'd1', 'd2'], alias="t2")
    def mapper():
        sql = select_map("mytable")
        return sql

    s2 = mapper()
    print('s2:  {}'.format(s2))

In [42]:
test_SelectMap_and_Map()

s1:  select id,start_date,len,split(y) as y from (select id,start_date,len,regexp_replace(regexp_replace(y, '\\]', ''), '\\[', '') as y from mytable WHERE type='ttt' and key='sales') t1
s2:  select id,start_date,len,concat(',', d1, d2) from (select id,start_date,len,d1,d2 from (select id,start_date,len,split(y) as y from (select id,start_date,len,regexp_replace(regexp_replace(y, '\\]', ''), '\\[', '') as y from mytable WHERE type='ttt' and key='sales') t1) t2) t2


# 参考

1. [py装饰器](https://github.com/WallaceLiu/py-decorator)